## NHL API

In [440]:
import requests as re
import json
from IPython.display import JSON
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import datetime

#### Below Cells will create DataFrame containing Corsi and Fenwick Stats, indexed by GameID

In [405]:
game = pd.read_csv('../Data/game.csv')

In [444]:
game_list_full = game.game_id.unique()

In [ ]:
# Run before main loop
game_dict = get_game_data(game_list_full)
master_dict = {}
columns=['corsi_for', 'corsi_against', 'cf_pct', 'ca_pct', 'fenwick_for', 'fenwick_against', 'ff_pct', 'fa_pct']
i = 0

### Execute Main API Call
    - The data from this call is stored already in the data folder, so don't run this unless you want to

In [ ]:
### Main Harness ###
for key, value in loaded_dict.items():
    
    if value == 0:
        
        continue
    
    # Create dataframe of chronological game plays
    play_df = get_play_df(value)
    
    # Create timestamp column representing gametime in seconds
    try:
        play_df['timestamp'] = play_df.apply(lambda x: get_seconds(x.time_of_event, x.period), axis=1)
    except Exception as e:
        
        print('\n======EXCEPTION=====')
        print(e)
        
        continue
    
    # Calculate shots based on even strength play
    home_shots, away_shots, home_blocked_shots, away_blocked_shots = count_all_shots(play_df)
    
    # Calculate Fenwick and Corsi stats
    stats = calc_advanced_stats(home_shots, away_shots, home_blocked_shots, away_blocked_shots)
    
    # Create dictionary holding the advanced stats
    test_dict = {}
    for i in range(len(stats)):
        test_dict[columns[i]] = stats[i]
    
    # Saving checkpoint
    if i % 3000 == 0:
        filename = 'game_df_checkpoint.sav'
        pickle.dump(master_dict, open(filename, 'wb')) 
    
    # Append dictionary with 1 game stats to master dict
    master_dict[key] = test_dict
    
    i += 1

advanced_stats_pt2 = pd.DataFrame(master_dict)
advanced_stats_pt2 = advanced_stats_pt2.T

# Saving checkpoint
pickle.dump(advanced_stats_final, open('../Data/full_advanced_stats.sav', 'wb')) 